# Pipeline As A Notebook

In [ ]:
!pip install -r ../requirements.txt

## Download Required Files

Download the required files and place them in th configured location below.

#### Hud Zip to Tract Crosswalk Dataset  
2020 and 2022 crosswalk files are also included in the repo.  
URL:  
Expected Location: `{download_directory}/crosswalk/zip_tract/`

#### SVI Dataset
URL:  
Expected Location: `{download_directory}/svi/`

#### Zillow HVI Dataset
URL:  
Expected Location: `{download_directory}/zillow`


In [3]:
download_directory = "../chosen_data"
year = 2020

## Database Connection

In [4]:
import duckdb

# create a connection to a file called 'test_database.db'
con = duckdb.connect("../data/database_jupyter.db")

## Create Tables
If they do not exist create the expected tables

Would need to include code for table migration/translation when new SVI datasets are downloaded

We will not precreate Zillows Bronze table, as those will be overwrites exclusively and then need to be pivoted. 

In [5]:
con.sql("SHOW TABLES")

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

In [ ]:
con.close()

In [2]:
import sys  
sys.path.insert(1, '../zillow_project/zillow_project/')



In [ ]:
import entry
import os

print(os.getcwd()) 
# May need to set this to the specific directory of the project depending on if the configuration uses relative or exact paths
os.chdir("../")
print(os.getcwd()) 
sys.argv[1]="--pipeline_config=./2022_pipeline.yaml"
entry.main()

In [13]:
import duckdb 

print(os.getcwd()) 
# create a connection to a file called 'test_database.db'
con = duckdb.connect("./data/medallion.db")
con.sql("SHOW TABLES")

d:\NASA_Interview_Fresh\housing_data


┌──────────────────────────┐
│           name           │
│         varchar          │
├──────────────────────────┤
│ svi                      │
│ tracts_per_zip           │
│ zhvi_sfr_zip             │
│ zhvi_sfr_zip_with_svi    │
│ zhvi_sfr_zip_with_tracts │
│ zip_tract                │
└──────────────────────────┘

In [14]:
con.close()